<a href="https://colab.research.google.com/github/camifezz/OrganizacionDeDatos-Tp2/blob/main/Random_Forest_binary_encoding_con_todas_las_columnas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalación de librerias

In [ ]:
!pip install PyDrive

In [ ]:
pip install --upgrade category_encoders

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import category_encoders as ce

from google.colab import files

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
id_train_labels = '1o8fEed65R8219m5iOJoZgvHXaTGwWE4H'
id_train_values = "1RCoa3FWbpcHVC6imhaQQFQEAWNW0qkh0" 
id_test_values = '1xBGPZbt3BYxe1OOYGMJuGnXe_pEqFZ1a'
id_submission_format = "1ei8BhsAAhZpZ8sRqEOVGl12y0JAPdXGV"

downloaded_train_labels = drive.CreateFile({'id': id_train_labels})
downloaded_train_values = drive.CreateFile({'id': id_train_values})
downloaded_test_values = drive.CreateFile({'id': id_test_values})
downloaded_submission_format = drive.CreateFile({'id': id_submission_format})

downloaded_train_labels.GetContentFile('train_labels.csv')
downloaded_train_values.GetContentFile('train_values.csv')
downloaded_test_values.GetContentFile('test_values.csv')
downloaded_submission_format.GetContentFile('submission_format.csv')

In [6]:
train_values = pd.read_csv('train_values.csv', dtype={
    'building_id': 'int64', 'geo_level_1_id': 'int64',
    'geo_level_2_id': 'int64', 'geo_level_3_id': 'int64',
    'count_floors_pre_eq': 'int8', 'age': 'int32',
    'area_percentage': 'int32', 'height_percentage': 'int32',
    'land_surface_condition': 'category', 'foundation_type': 'category',
    'roof_type': 'category', 'ground_floor_type': 'category', 'other_floor_type': 'category',
    'position': 'category', 'plan_configuration': 'category', 'has_superstructure_adobe_mud': 'int8',
    'has_superstructure_mud_mortar_stone': 'int8', 'has_superstructure_stone_flag': 'int8',
    'has_superstructure_cement_mortar_stone': 'int8', 'has_superstructure_mud_mortar_brick': 'int8',
    'has_superstructure_cement_mortar_brick': 'int8', 'has_superstructure_timber': 'int8',
    'has_superstructure_bamboo': 'int8', 'has_superstructure_rc_non_engineered': 'int8',
    'has_superstructure_rc_engineered': 'int8', 'has_superstructure_other': 'int8',
    'has_superstructure_other': 'int8', 'legal_ownership_status': 'category',
    'has_secondary_use': 'bool', 'has_secondary_use_agriculture': 'bool',
    'has_secondary_use_hotel': 'bool', 'has_secondary_use_rental': 'bool',
    'has_secondary_use_institution': 'bool', 'has_secondary_use_school': 'bool',
    'has_secondary_use_industry': 'bool', 'has_secondary_use_health_post': 'bool',
    'has_secondary_use_gov_office': 'bool', 'has_secondary_use_use_police': 'bool',
    'has_secondary_use_other': 'bool', 'count_families': 'int8'
})
train_labels = pd.read_csv('train_labels.csv', dtype={
    'damage_grade': 'int16'
})

test_values = pd.read_csv('test_values.csv', dtype={
    'building_id': 'int64', 'geo_level_1_id': 'int64',
    'geo_level_2_id': 'int64', 'geo_level_3_id': 'int64',
    'count_floors_pre_eq': 'int8', 'age': 'int32',
    'area_percentage': 'int32', 'height_percentage': 'int32',
    'land_surface_condition': 'category', 'foundation_type': 'category',
    'roof_type': 'category', 'ground_floor_type': 'category', 'other_floor_type': 'category',
    'position': 'category', 'plan_configuration': 'category', 'has_superstructure_adobe_mud': 'int8',
    'has_superstructure_mud_mortar_stone': 'int8', 'has_superstructure_stone_flag': 'int8',
    'has_superstructure_cement_mortar_stone': 'int8', 'has_superstructure_mud_mortar_brick': 'int8',
    'has_superstructure_cement_mortar_brick': 'int8', 'has_superstructure_timber': 'int8',
    'has_superstructure_bamboo': 'int8', 'has_superstructure_rc_non_engineered': 'int8',
    'has_superstructure_rc_engineered': 'int8', 'has_superstructure_other': 'int8',
    'has_superstructure_other': 'int8', 'legal_ownership_status': 'category',
    'has_secondary_use': 'bool', 'has_secondary_use_agriculture': 'bool',
    'has_secondary_use_hotel': 'bool', 'has_secondary_use_rental': 'bool',
    'has_secondary_use_institution': 'bool', 'has_secondary_use_school': 'bool',
    'has_secondary_use_industry': 'bool', 'has_secondary_use_health_post': 'bool',
    'has_secondary_use_gov_office': 'bool', 'has_secondary_use_use_police': 'bool',
    'has_secondary_use_other': 'bool', 'count_families': 'int8'
})

In [7]:
labels_and_values = pd.merge(train_labels, train_values, how='inner', on='building_id')

In [9]:
data = labels_and_values.loc[labels_and_values["age"] <995]

In [10]:
prediction = data['damage_grade']





##Binary encoding con todas las cols

In [11]:
encoder= ce.BinaryEncoder(cols=['land_surface_condition','foundation_type','roof_type','ground_floor_type','position',"other_floor_type","plan_configuration"],return_df=True)

In [12]:
data_encoded=encoder.fit_transform(data) 

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
data_encoded_test=encoder.fit_transform(test_values) 

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
data_encoded

In [14]:
features = data_encoded[['geo_level_1_id',
    'geo_level_2_id', 'geo_level_3_id','area_percentage','age',"height_percentage",'has_superstructure_adobe_mud',
    'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag','has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick',
    'has_superstructure_cement_mortar_brick', 'has_superstructure_timber','has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
    'has_superstructure_rc_engineered', 'has_superstructure_other',"land_surface_condition_0",	"land_surface_condition_1",	"land_surface_condition_2",
    	"foundation_type_0",	"foundation_type_1",	"foundation_type_2",	"foundation_type_3",	"roof_type_0",	"roof_type_1",	"roof_type_2"	,
      "ground_floor_type_0",	"ground_floor_type_1",	"ground_floor_type_2",	"ground_floor_type_3",	
      	"position_0",	"position_1",	"position_2","other_floor_type_0",	"other_floor_type_1",	"other_floor_type_2",
        "plan_configuration_0",	"plan_configuration_1",	"plan_configuration_2",	"plan_configuration_3",	"plan_configuration_4"
        ]]

In [ ]:
features_test_values = data_encoded_test[['geo_level_1_id',
    'geo_level_2_id', 'geo_level_3_id','area_percentage','age',"height_percentage",'has_superstructure_adobe_mud',
    'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag','has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick',
    'has_superstructure_cement_mortar_brick', 'has_superstructure_timber','has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
    'has_superstructure_rc_engineered', 'has_superstructure_other',"land_surface_condition_0",	"land_surface_condition_1",	"land_surface_condition_2",
    	"foundation_type_0",	"foundation_type_1",	"foundation_type_2",	"foundation_type_3",	"roof_type_0",	"roof_type_1",	"roof_type_2"	,
      "ground_floor_type_0",	"ground_floor_type_1",	"ground_floor_type_2",	"ground_floor_type_3",	
      	"position_0",	"position_1",	"position_2","volume_percentage","other_floor_type_0",	"other_floor_type_1",	"other_floor_type_2",
        "plan_configuration_0",	"plan_configuration_1",	"plan_configuration_2",	"plan_configuration_3",	"plan_configuration_4"
        ]]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(features, prediction, test_size=0.2, random_state=1)

In [17]:
modelo_RF = RandomForestClassifier(random_state=1)
modelo_RF.fit(X_train, y_train) #entrenamos el modelo
prediccion = modelo_RF.predict(X_test)

In [ ]:
prediccion_test_values = modelo_RF.predict(features_test_values)

In [18]:
f1_score(prediccion, y_test, average='micro')

0.7112821403082383

In [ ]:
plt.subplots(figsize=(20,15))
plt.bar(X_train.columns, modelo_RF.feature_importances_, color="grey")
plt.xlabel('Features')
plt.ylabel('Importancia')
plt.title('Importancia de Features RF Binary Encoding')
plt.xticks(rotation=90)

plt.show()

In [27]:
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')

In [ ]:
my_submission = pd.DataFrame(data = prediccion_test_values,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [ ]:
my_submission.to_csv('submission.csv')

In [ ]:
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>